In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf 
from pathlib import Path 
import os 

In [9]:
DatasetPath = Path('E:\dataset_folder\data_skintone')


In [ ]:
Data = {}

for folder in os.listdir(DatasetPath):
    folder_path = os.path.join(DatasetPath, folder)
    if not folder_path:
        continue 
    
    img_list= []
    label = folder
    for img in os.listdir(folder_path):
        img_path =os.path.join(folder_path, img)
        img_list.append(img_path)

    Data[label] = img_list


4

In [32]:
print(f'Length of light is {len(Data["light"])} \nLength of mid-light is {len(Data["mid-light"])} \nLength of mid-dark is {len(Data["mid-dark"])} \nLength of dark is {len(Data["dark"])}')


Length of light is 9769 
Length of mid-light is 6844 
Length of mid-dark is 10576 
Length of dark is 8640


In [41]:
labelMap = {}
for pos,i in enumerate(list(Data.keys())):
    labelMap[i] = pos
labelMap

{'dark': 0, 'light': 1, 'mid-dark': 2, 'mid-light': 3}

In [43]:
import cv2 
import numpy as np 

X= []
Y= []

for label, img_list in Data.items():
    for img in img_list:
        img = cv2.imread(img)
        if  img is None:
            continue
        img = cv2.resize(img, (224, 224))
        img = cv2.cvtColor(img , cv2.COLOR_BGR2RGB)
        img = img/255.0

        X.append(img)
        Y.append(labelMap[label])

X = np.array(X)
Y = np.array(Y)


KeyboardInterrupt: 

In [39]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y ,random_state=42, test_size = 0.2)

NameError: name 'X' is not defined

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define CNN model
model = Sequential([
    # 1st Convolution Block
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),

    # 2nd Convolution Block
    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),

    # 3rd Convolution Block
    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),

    # Flatten and Dense Layers
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),

    Dense(len(labelMap), activation='softmax')  # Output layer (num_classes = number of skin tones)
])


In [ ]:
# Compile Model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
# Show model summary
model.summary()

# Define Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# Train model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=25,
    batch_size=32,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)


In [ ]:
# Evaluate model
loss, acc = model.evaluate(X_test, y_test)
print(f"✅ Test Accuracy: {acc*100:.2f}%")